# Notebook Transaksi Mesin Ramayana Makmur Sentosa Playzone Tahun 2024 - 2025

## Data Wrangling

In [196]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px  # Kita pakai Plotly agar grafik interaktif (cocok buat Streamlit)
import re
import matplotlib.ticker as ticker
import plotly.graph_objects as go
from rapidfuzz import fuzz

Load Data

In [197]:
# Konfigurasi tampilan pandas
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# LOAD DATA
# Pastikan file ini adalah hasil output dari script terakhir Anda
file_path = "REKAP_DATA_MESIN_FULL.xlsx"
df = pd.read_excel(file_path)

print(f"Data Loaded: {df.shape[0]} baris, {df.shape[1]} kolom")
df.head()

Data Loaded: 153452 baris, 19 kolom


,Center,Kode Game,Game Title,Keterangan,Kategori Game,Jumlah Diaktifkan,Kredit yg Digunakan,Bonus yg Digunakan,Main Gratis/Koin Masuk,QRIS,Total,Tiket/Koin Keluar,Mercy Ticket,Payout,Avg. Ticket/Coin Out Per Play/Coin In,Bulan,Tahun,Asal_Folder,Nama_File_Asal
0,Tamini Square,CS01,CEK SALDO,CEK SALDO #1,Cek Saldo,92.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Agustus,2024,data-mesin,Laporan_Agustus_2024.xlsx
1,Tamini Square,CS02,CEK SALDO,CEK SALDO #2,Cek Saldo,168.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Agustus,2024,data-mesin,Laporan_Agustus_2024.xlsx
2,Tamini Square,NEW 29 /10/16,KIDDIE LAND,KIDDIE LAND SEPUASNYA,KIDDIE LAND,11.00,"385,000.00",0.00,0.00,0.00,"385,000.00",0.00,0.00,0.00,0.00,Agustus,2024,data-mesin,Laporan_Agustus_2024.xlsx
3,Tamini Square,4/06/2024,CRAZY HORSE,CRAZY HORSE # 5,Kiddy Ride,22.00,"70,350.00","6,650.00",0.00,0.00,"77,000.00",0.00,0.00,0.00,0.00,Agustus,2024,data-mesin,Laporan_Agustus_2024.xlsx
4,Tamini Square,04/6/2024,CRAZY HORSE,CRAZY HORSE # 6,Kiddy Ride,9.00,"30,000.00","1,500.00",0.00,0.00,"31,500.00",0.00,0.00,0.00,0.00,Agustus,2024,data-mesin,Laporan_Agustus_2024.xlsx


Data Understanding : 
* Center = Nama toko
* Kode Game = Kode mesin individual
* Game Title = Nama mesin
* Keterangan = Nama mesin + jumlah player
* Kategori game = Jenis mesin
* Jumlah diaktifkan = Frekuensi mesin dimainkan per bulan
* Kredit yang digunakan = Jumlah uang yang dikeluarkan pemain pada suatu mesin per bulan
* Bonus yang digunakan = 
* Main Gratis/Koin Masuk = 
* QRIS = 
* Total = 
* Tiket/Koin Keluar = 
* Mercy Ticket = 
* Payout = 
* Avg. Ticket/Coin Out Per Play/Coin In = 
* Bulan = Periode bulan transaksi mesin
* Tahun = Periode tahun transaksi mesin
* Asal_Folder = Asal folder lokal data discraping
* Nama_File_Asal = Asal file lokal data discraping

Assesing Data

In [198]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153452 entries, 0 to 153451
Data columns (total 19 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   Center                                 147930 non-null  object 
 1   Kode Game                              147930 non-null  object 
 2   Game Title                             147930 non-null  object 
 3   Keterangan                             147930 non-null  object 
 4   Kategori Game                          147930 non-null  object 
 5   Jumlah Diaktifkan                      147930 non-null  float64
 6   Kredit yg Digunakan                    147930 non-null  float64
 7   Bonus yg Digunakan                     147930 non-null  float64
 8   Main Gratis/Koin Masuk                 147930 non-null  float64
 9   QRIS                                   147930 non-null  float64
 10  Total                                  147930 non-null  

Dataframe df memiliki 147930 - 153452 entry data. Terdapat . Terdapat 5.522 data yang hilang. Untuk kolom tahun dan bulan masih berbentuk integer dan object sehingga perlu preprocessing menjadi datetime agar dapat difilter nantinya.

*Pemeriksaan parameter statistik*

In [199]:
df.describe()

,Jumlah Diaktifkan,Kredit yg Digunakan,Bonus yg Digunakan,Main Gratis/Koin Masuk,QRIS,Total,Tiket/Koin Keluar,Mercy Ticket,Payout,Avg. Ticket/Coin Out Per Play/Coin In,Tahun
count,"147,930.00","147,930.00","147,930.00","147,930.00","147,930.00","147,930.00","147,930.00","147,930.00","147,930.00","147,930.00","153,452.00"
mean,271.74,"1,274,711.13","397,893.83",1.56,0.00,"1,673,101.31","22,666.32",0.00,0.17,30.75,"2,024.50"
std,409.03,"3,261,281.89","890,596.39",23.90,0.00,"3,691,426.39","2,402,944.66",0.00,12.80,"2,254.23",0.50
min,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,024.00"
25%,57.00,"174,900.00","33,300.00",0.00,0.00,"240,100.00",0.00,0.00,0.00,0.00,"2,024.00"
50%,144.00,"514,500.00","133,700.00",0.00,0.00,"705,600.00",0.00,0.00,0.00,0.00,"2,025.00"
75%,321.00,"1,330,387.50","403,000.00",0.00,0.00,"1,803,200.00",0.00,0.00,0.00,0.00,"2,025.00"
max,"10,491.00","323,749,776.00","37,761,998.00","2,994.00",0.00,"323,749,776.00","551,634,620.00",0.00,"1,900.88","263,193.13","2,025.00"


* Tidak ada outlier yang signifikan
* Kolom QRIS dan Mercy Ticket kosong sama sekali sehingga bisa didrop
* kolom Main Gratis/Koin Masuk,  Tiket/Koin Keluar, Payout, dan Avg. Ticket/Coin Out Per Play/Coin In banyak value yang kosong sehingga bisa didrop.

## Data Cleaning & Preprocessing

Drop kolom yang seluruhnya kosong dan mayoritas kosong karena sedikit informasinya. Serta asal folder dan file karena kurang informarif untuk analisis.

In [200]:
cols_to_drop = [
    'QRIS', 
    'Mercy Ticket', 
    'Main Gratis/Koin Masuk', 
    'Tiket/Koin Keluar', 
    'Payout', 
    'Avg. Ticket/Coin Out Per Play/Coin In',
    'Asal_Folder',
    'Nama_File_Asal'

]
df.drop(columns=cols_to_drop, axis=1, inplace=True, errors='ignore')
display(df.head())

,Center,Kode Game,Game Title,Keterangan,Kategori Game,Jumlah Diaktifkan,Kredit yg Digunakan,Bonus yg Digunakan,Total,Bulan,Tahun
0,Tamini Square,CS01,CEK SALDO,CEK SALDO #1,Cek Saldo,92.00,0.00,0.00,0.00,Agustus,2024
1,Tamini Square,CS02,CEK SALDO,CEK SALDO #2,Cek Saldo,168.00,0.00,0.00,0.00,Agustus,2024
2,Tamini Square,NEW 29 /10/16,KIDDIE LAND,KIDDIE LAND SEPUASNYA,KIDDIE LAND,11.00,"385,000.00",0.00,"385,000.00",Agustus,2024
3,Tamini Square,4/06/2024,CRAZY HORSE,CRAZY HORSE # 5,Kiddy Ride,22.00,"70,350.00","6,650.00","77,000.00",Agustus,2024
4,Tamini Square,04/6/2024,CRAZY HORSE,CRAZY HORSE # 6,Kiddy Ride,9.00,"30,000.00","1,500.00","31,500.00",Agustus,2024


Pemeriksaan duplikasi

In [201]:
print("Jumlah data duplikat dari df:", df.duplicated().sum())

Jumlah data duplikat dari df: 5521


Ada data duplikat, maka perlu dihandle.

In [202]:
df_clean = df.drop_duplicates()

Cek kembali duplikat.

In [203]:
print("Jumlah data duplikat dari df setelah dihandle:", df_clean.duplicated().sum())

Jumlah data duplikat dari df setelah dihandle: 0


Lihat null di setiap kolom

In [204]:
# Menghitung jumlah null di setiap kolom
print(df.isna().sum())

Center                 5522
Kode Game              5522
Game Title             5522
Keterangan             5522
Kategori Game          5522
Jumlah Diaktifkan      5522
Kredit yg Digunakan    5522
Bonus yg Digunakan     5522
Total                  5522
Bulan                     0
Tahun                     0
dtype: int64


Lihat presentase missing values

In [205]:
# Menampilkan jumlah dan persentase
missing_data = df.isna().sum()
total_rows = len(df)

percent_missing = (missing_data / total_rows) * 100

# Gabungkan jadi satu tabel agar mudah dibaca
missing_df = pd.DataFrame({
    'Total Missing': missing_data,
    'Persentase (%)': percent_missing
})

# Tampilkan hanya kolom yang ada missing values-nya (urut dari terbesar)
print(missing_df[missing_df['Total Missing'] > 0].sort_values('Persentase (%)', ascending=False))

                     Total Missing  Persentase (%)
Center                        5522            3.60
Kode Game                     5522            3.60
Game Title                    5522            3.60
Keterangan                    5522            3.60
Kategori Game                 5522            3.60
Jumlah Diaktifkan             5522            3.60
Kredit yg Digunakan           5522            3.60
Bonus yg Digunakan            5522            3.60
Total                         5522            3.60


Cek baris yang kosong

In [206]:
# Menampilkan baris yang memiliki setidaknya satu kolom kosong
baris_kosong = df[df.isna().any(axis=1)]

display(baris_kosong.head())

,Center,Kode Game,Game Title,Keterangan,Kategori Game,Jumlah Diaktifkan,Kredit yg Digunakan,Bonus yg Digunakan,Total,Bulan,Tahun
32583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Desember,2025
32584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Desember,2025
32585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Desember,2025
32586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Desember,2025
32587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Desember,2025


Drop missing value

In [207]:
# Kolom yang ingin di drop
df.dropna(subset=['Center', 'Kode Game', 'Game Title'], inplace=True)

# Reset index agar urut kembali
df.reset_index(drop=True, inplace=True)

Cek kembali statistik umum setelah cleaning data

In [208]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 147931 entries, 0 to 153451
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Center               147930 non-null  object 
 1   Kode Game            147930 non-null  object 
 2   Game Title           147930 non-null  object 
 3   Keterangan           147930 non-null  object 
 4   Kategori Game        147930 non-null  object 
 5   Jumlah Diaktifkan    147930 non-null  float64
 6   Kredit yg Digunakan  147930 non-null  float64
 7   Bonus yg Digunakan   147930 non-null  float64
 8   Total                147930 non-null  float64
 9   Bulan                147931 non-null  object 
 10  Tahun                147931 non-null  int64  
dtypes: float64(4), int64(1), object(6)
memory usage: 13.5+ MB


Terdapat pengurangan banyaknya data menjadi 147931

**Cleaning data waktu agar datetime**

Mapping Bulan dan Tahun ke Tanggal (datetime) agar mudah dikategorikan

In [209]:
# Standardisasi Nama Bulan ---
# Pastikan nama bulan konsisten (Title Case: Januari, bukan januari)
# Menggunakan df_clean
df_clean['Bulan'] = df_clean['Bulan'].astype(str).str.title()

# Mapping Bulan Indonesia ke Angka (Untuk bikin tanggal)
map_bulan = {
    'Januari': 1, 'Februari': 2, 'Maret': 3, 'April': 4,
    'Mei': 5, 'Juni': 6, 'Juli': 7, 'Agustus': 8,
    'September': 9, 'Oktober': 10, 'November': 11, 'Desember': 12
}

df_clean['Bulan_Angka'] = df_clean['Bulan'].map(map_bulan)

# Membuat Kolom Tanggal (Date) ---
# Kita set tanggal 1 sebagai default karena data kita bulanan
# Menggunakan df_clean.Tahun dan df_clean.Bulan_Angka
df_clean['Tanggal'] = pd.to_datetime(dict(year=df_clean.Tahun, month=df_clean.Bulan_Angka, day=1))

# Filter Data Aneh (Optional) ---
# Contoh: Hapus jika Total_Sales negatif (jika perlu)
# df_clean = df_clean[df_clean['Total_Sales'] >= 0] 

print("Rentang Data:", df_clean['Tanggal'].min(), "sampai", df_clean['Tanggal'].max())
df_clean.info()

Rentang Data: 2024-01-01 00:00:00 sampai 2025-12-01 00:00:00
<class 'pandas.core.frame.DataFrame'>
Index: 147931 entries, 0 to 153451
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Center               147930 non-null  object        
 1   Kode Game            147930 non-null  object        
 2   Game Title           147930 non-null  object        
 3   Keterangan           147930 non-null  object        
 4   Kategori Game        147930 non-null  object        
 5   Jumlah Diaktifkan    147930 non-null  float64       
 6   Kredit yg Digunakan  147930 non-null  float64       
 7   Bonus yg Digunakan   147930 non-null  float64       
 8   Total                147930 non-null  float64       
 9   Bulan                147931 non-null  object        
 10  Tahun                147931 non-null  int64         
 11  Bulan_Angka          147931 non-null  int64         
 12  Tanggal         

C:\Users\ACER\AppData\Local\Temp\ipykernel_11904\3290139826.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Bulan'] = df_clean['Bulan'].astype(str).str.title()
C:\Users\ACER\AppData\Local\Temp\ipykernel_11904\3290139826.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Bulan_Angka'] = df_clean['Bulan'].map(map_bulan)
C:\Users\ACER\AppData\Local\Temp\ipykernel_11904\3290139826.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usi

**Filter center yang aktif saja dan mapping standarisasi nama toko**

Map Toko Aktif

In [210]:
# Key (Kiri) = Nama di Center
# Value (Kanan) = Nama Baru Standar
mapping_center = {
    # --- AREA R ---
    "Abepura (R-91)": "R091 Abepura",
    "Samarinda (R-94) RMS": "R094 Samarinda",
    "Panam (R-96)": "R096 Panam",
    "Kediri (R-98) RMS": "R098 Kediri",
    "Garut (R-99) RMS": "R099 Garut",
    "Sabar Subur": "R135 SS Tangerang",       
    "Cimone (R-26) IFS": "R026 Cimone",
    "Plaza Sukaramai Pekanbaru (R-56)": "R056 Pekanbaru",
    "Cirebon (R-100)": "R100 Cirebon",
    "Rajabasa Lampung": "R101 Rajabasa",
    "Kebayoran Lama (R-102) IFS": "R102 Kebayoran Lama",
    "Sorong (R-103)": "R103 Sorong",
    "Parung": "R105 Parung",
    "Cibadak (R-106)": "R106 Cibadak",
    "Pekalongan (R-110)": "R110 Pekalongan",
    "Yasmin Bogor": "R114 Bogor Yasmin",
    "Dinoyo": "R119 Dinoyo",
    "Tajur (R-121)": "R121 Tajur",
    "Cikupa": "R124 Cikupa",
    "Jatinegara": "R125 Jatinegara",
    "Timika (R-138)": "R138 Timika",
    "Dewi Sartika Bogor": "R015 Dewi Sartika",

    # --- AREA RB ---
    "Sukabumi (RB-05) RMS": "RB05 Sukabumi",
    "Cimahi (RB-12)": "RB12 Cimahi",
    "Gresik (RB-16) RMS": "RB16 Gresik",
    "Denpasar (RB-20) RMS": "RB20 Denpasar",
    "Sidoarjo (RB-27) RMS": "RB27 Sidoarjo",
    "Kudus (RB-28)": "RB28 Kudus",
    "Panbil Batam (RB-30) RMS": "RB30 Panbil",
    "Malang (RB-M) RMS": "RBM Malang",
    "Krian (RB-23)": "RB23 Krian",

    # --- AREA ZD ---
    "Ubud Bali": "ZD15 Bali Batu Bulan",
    "Wonosari": "ZD25 Wonosari",
    "Ponorogo (ZD-44)": "ZD44 Ponorogo",
    "Lotte Bintaro (ZD-46)": "ZD46 Lotte Bintaro",
    "Payakumbuh": "ZD39 Payakumbuh",
    "Mesra Samarinda (ZD-37)": "ZD37 Samarinda"
}

Filter dan Rename

In [211]:
# Bersihkan spasi berlebih agar pencocokan akurat
df['Center'] = df['Center'].astype(str).str.strip()

# Langkah A: Filter (Hanya ambil data yang Center-nya ada di daftar mapping)
df_clean = df[df['Center'].isin(mapping_center.keys())].copy()

# Langkah B: Rename (Ubah nama lama jadi nama baru)
df_clean['Center'] = df_clean['Center'].map(mapping_center)

# ================= 3. CEK HASIL =================
print(f"Data Awal: {len(df)} baris")
print(f"Data Setelah Filter Center Aktif: {len(df_clean)} baris")
print("\nDaftar Center Aktif (Standardized):")
print(df_clean['Center'].unique())

# Update ke dataframe utama
df = df_clean

Data Awal: 147930 baris
Data Setelah Filter Center Aktif: 63467 baris

Daftar Center Aktif (Standardized):
['R102 Kebayoran Lama' 'R125 Jatinegara' 'ZD46 Lotte Bintaro'
 'R015 Dewi Sartika' 'R105 Parung' 'R121 Tajur' 'R106 Cibadak'
 'R099 Garut' 'R124 Cikupa' 'RB05 Sukabumi' 'R135 SS Tangerang'
 'R026 Cimone' 'R101 Rajabasa' 'ZD39 Payakumbuh' 'RB30 Panbil'
 'R096 Panam' 'R056 Pekanbaru' 'R094 Samarinda' 'R103 Sorong'
 'R091 Abepura' 'R138 Timika' 'RB28 Kudus' 'R110 Pekalongan'
 'ZD25 Wonosari' 'R119 Dinoyo' 'RBM Malang' 'RB27 Sidoarjo' 'RB16 Gresik'
 'RB23 Krian' 'R098 Kediri' 'RB20 Denpasar' 'ZD15 Bali Batu Bulan'
 'R114 Bogor Yasmin' 'RB12 Cimahi' 'R100 Cirebon' 'ZD37 Samarinda'
 'ZD44 Ponorogo']


Coba Mapping 'Game Title'

In [212]:
import re

def normalize_game_title(text: str) -> str:
    if not isinstance(text, str):
        return text

    text = text.upper()
    text = re.sub(r"\s+", " ", text)      # multiple space → single
    text = text.strip()
    return text


In [213]:
def pre_clean(text):
    text = normalize_game_title(text)
    text = re.sub(r"\s?#\d+|\s?\([A-Z]\)", "", text)
    text = re.sub(r"\b(SD|TWIN|SINGLE|DELUXE|DX|JR|MINI|BIG)\b", "", text)
    return text.strip()


In [214]:
game_title_map = {
    # ================= AIR HOCKEY =================
    "JUMBO AIR HOCKEY 2000": "JUMBO AIR HOCKEY",
    "PAIR MATCH HOCKEY": "JUMBO AIR HOCKEY",

    # ================= RAINBOW =================
    "RAINBOW SPINNER": "RAINBOW SPINNER",
    "RAINBOW SPINER": "RAINBOW SPINNER",
    "RANBOW SPINNER": "RAINBOW SPINNER",
    "RAINBOW SPINER": "RAINBOW SPINNER",
    "RAINBOW": "RAINBOW",

    # ================= SPEED DRIVER =================
    "SPEED DRIVER": "SPEED DRIVER",
    "SPEED DRIVER 3 SD TWIN": "SPEED DRIVER",
    "CRAZY SPEED": "SPEED DRIVER",
    "CRAZY SPEED SD SINGLE": "SPEED DRIVER",
    "CRAZY SPEED TWIN": "SPEED DRIVER",

    # ================= SPEED RIDER =================
    "SPEED RIDER": "SPEED RIDER",
    "SPEED RIDER 2 TWIN": "SPEED RIDER",
    "SPEED RIDER 3": "SPEED RIDER",
    "SPEED RIDER 3D": "SPEED RIDER",

    # ================= SPEEDING UP =================
    "SPEEDING UP": "SPEEDING UP",

    # ================= MAXIMUM TUNE =================
    "MAXIMUM TUNE": "MAXIMUM TUNE",

    # ================= RACING TUNED =================
    "RACING TUNED": "RACING TUNED",
    "R TUNED": "RACING TUNED",
    "R-TUNED RACING": "RACING TUNED",

    # ================= KERETA =================
    "KERETA KENCANA": "CLASSIC WAGOON",
    "KERETA GOYANG": "CLASSIC WAGOON",
    "KERETA WAHANA": "CHOOCHOO TRAIN",

    # ================= ZAMPERLA =================
    "ZAMPERLA HORSE": "ZAMPERLA RIDE",
    "ZAMPERLA UNTA": "ZAMPERLA RIDE",

    # ================= POP IT WIN =================
    "POP IT WIN": "POP IT WIN",
    "POP IT & WIN": "POP IT WIN",
    "POP IT N WIN": "POP IT WIN",
    "POT IT WIN": "POP IT WIN",
    "PO IT WIN": "POP IT WIN",
}


In [215]:
df["Game Title Clean"] = (
    df["Game Title"]
    .apply(normalize_game_title)
    .replace(game_title_map)
)


In [216]:
df.groupby("Game Title Clean")["Game Title"].nunique() \
  .sort_values(ascending=False) \
  .head(20)


Game Title Clean
POP IT WIN           6
SPEED DRIVER         5
RAINBOW SPINNER      4
SPEED RIDER          4
CLASSIC WAGOON       3
JUMBO AIR HOCKEY     3
POWER TRUCK          3
RACING TUNED         3
SHARK PARK           3
DRAGON ORB           2
HOCKEY 2000          2
BABY BALON HOCKEY    2
BIG BIKE             2
DINO WHEEL           2
KIDDIE LAND          2
KUDA PONI            2
GOLDEN GEAR          2
GLOBAL ADVENTURE     2
FISHING MASTER       2
MESIN BARU           2
Name: Game Title, dtype: int64

In [217]:
conflict = (
    df.assign(
        GT_NORM=df["Game Title"].apply(normalize_game_title),
        GT_CLEAN=lambda x: x["GT_NORM"].replace(game_title_map)
    )
    .groupby("GT_CLEAN")["GT_NORM"]
    .nunique()
    .reset_index(name="jumlah_variasi")
    .query("jumlah_variasi > 1")
    .sort_values("jumlah_variasi", ascending=False)
)

conflict


,GT_CLEAN,jumlah_variasi
504,POP IT WIN,5
600,SPEED DRIVER,5
601,SPEED RIDER,4
104,CLASSIC WAGOON,3
524,RACING TUNED,3
312,JUMBO AIR HOCKEY,3
529,RAINBOW SPINNER,3
722,ZAMPERLA RIDE,2


In [218]:
detail = (
    df.assign(
        GT_NORM=df["Game Title"].apply(normalize_game_title),
        GT_CLEAN=lambda x: x["GT_NORM"].replace(game_title_map)
    )
    .query("GT_CLEAN in @conflict.GT_CLEAN")
    [["Game Title", "GT_NORM", "GT_CLEAN"]]
    .sort_values("GT_CLEAN")
)

detail


,Game Title,GT_NORM,GT_CLEAN
124433,CLASSIC WAGOON,CLASSIC WAGOON,CLASSIC WAGOON
124823,KERETA KENCANA,KERETA KENCANA,CLASSIC WAGOON
8447,KERETA KENCANA,KERETA KENCANA,CLASSIC WAGOON
112078,CLASSIC WAGOON,CLASSIC WAGOON,CLASSIC WAGOON
145002,CLASSIC WAGOON,CLASSIC WAGOON,CLASSIC WAGOON
...,...,...,...
83711,ZAMPERLA UNTA,ZAMPERLA UNTA,ZAMPERLA RIDE
51645,ZAMPERLA UNTA,ZAMPERLA UNTA,ZAMPERLA RIDE
116489,ZAMPERLA UNTA,ZAMPERLA UNTA,ZAMPERLA RIDE
121905,ZAMPERLA HORSE,ZAMPERLA HORSE,ZAMPERLA RIDE


In [219]:
def master_clean(text):
    text = normalize_game_title(text)

    text = re.sub(r"\s?#\d+|\s?\([A-Z]\)", "", text)

    if re.search(r"POP\s*I?T?\s*(N|&)?\s*WIN", text):
        return "POP IT WIN"

    if "CRAZY SPEED" in text or "SPEED DRIVER" in text:
        return "SPEED DRIVER"

    if "AIR HOCKEY" in text or "PAIR MATCH HOCKEY" in text:
        return "JUMBO AIR HOCKEY"

    if text in ["KERETA GOYANG", "KERETA KENCANA", "CLASSIC WAGON", "CLASSIC WAGOON"]:
        return "CLASSIC WAGOON"

    if re.search(r"R.?AINBOW\s*SPIN", text):
        return "RAINBOW SPINNER"

    if re.search(r"\bR\s*-?\s*TUNED\b", text) or "RACING TUNED" in text:
        return "RACING TUNED"

    if text in ["CHOOCHOO TRAIN", "KERETA WAHANA"]:
        return "CHOOCHOO TRAIN"

    if text.startswith("ZAMPERLA"):
        return "ZAMPERLA RIDE"

    return text


In [220]:
df["GT_FINAL"] = df["Game Title"].apply(master_clean)

df.groupby("GT_FINAL")["GT_FINAL"].nunique().sort_values(ascending=False)


GT_FINAL
2D MINI KIDDIE RIDE    1
AIR HOCKER 2000        1
AIR HOCKY              1
ALIEN                  1
ALLIENS ARMAGEDDON     1
                      ..
ZOMBIE OUTBREAK        1
ZOO MOVE               1
ZOOMOVE                1
ZOOMOVE SERVER         1
ZOOOMOVE               1
Name: GT_FINAL, Length: 672, dtype: int64

Dapatkan unique value untuk GT_FINAL

In [221]:
titles = sorted(df["GT_FINAL"].unique())

Fuzzy Similarity

In [222]:
rows = []

for i, a in enumerate(titles):
    for b in titles[i+1:]:
        score = fuzz.token_sort_ratio(a, b) / 100
        if score >= 0.85:   # threshold audit
            rows.append({
                "GT_A": a,
                "GT_B": b,
                "similarity": round(score, 2)
            })

audit_df = pd.DataFrame(rows).sort_values("similarity", ascending=False)
audit_df


,GT_A,GT_B,similarity
16,CHILDREN F1,F1 CHILDREN,1.00
68,GHOST OPERATION,OPERATION GHOST,1.00
81,HOCKEY MINI,MINI HOCKEY,1.00
37,DOG POUNDER,POUNDER DOG,1.00
24,CRANE MOBIL,MOBIL CRANE,1.00
...,...,...,...
164,SCHOOL BUS,SCHOOL RUSH,0.86
185,SWEET FRENZY,SWEETY FRENZY P1,0.86
181,SUPER BIKE,SURFER BIKE,0.86
204,TRLOLEY,TROLLEY,0.86


Ekspor Audit Excel

In [223]:
'''
output_path = "audit_game_title_similarity.xlsx"

audit_df.to_excel(
    output_path,
    index=False,
    sheet_name="Audit Similarity"
)

print(f"Audit berhasil diekspor ke {output_path}")
'''

'\noutput_path = "audit_game_title_similarity.xlsx"\n\naudit_df.to_excel(\n    output_path,\n    index=False,\n    sheet_name="Audit Similarity"\n)\n\nprint(f"Audit berhasil diekspor ke {output_path}")\n'

Baca Audit Merge Excel ke Dataframe

In [224]:
audit_df = pd.read_excel("audit_merge_map.xlsx")

audit_merge_map = dict(
    zip(
        audit_df["GT_B"].str.upper().str.strip(),
        audit_df["GT_A"].str.upper().str.strip()
    )
)


APPLY ke GT_FINAL

In [225]:
df["GT_FINAL"] = (
    df["GT_FINAL"]
    .str.upper()
    .str.strip()
    .replace(audit_merge_map)
)


Validasi WAJIB

In [226]:
set(audit_merge_map.keys()) & set(df["GT_FINAL"].unique())


{'BABY BALON HOCKEY',
 'DOLPHIN STAR',
 'DRIVING SCHOOL',
 'FANCY CUTTER',
 'MAXIMUM TUNE 3',
 'MAXIMUM TUNE 3 DX',
 'MAXIMUM TUNE 3 DX PLUS',
 'NAUGTHY BEANS',
 'PEASHOOTER',
 'PIRATES CANNON BALL',
 'PLATINUM HEIGHT',
 'POP IT WIN',
 'PUMP IT UP',
 'TOYS FACTORY',
 'TROLEY',
 'TROLLEY'}

Semua GT_A harus ADA

In [227]:
set(audit_merge_map.values()) - set(df["GT_FINAL"].unique())


set()

In [228]:
df.groupby("GT_FINAL")["Game Title"].nunique() \
  .sort_values(ascending=False) \
  .head(20)


GT_FINAL
CRAZY HORSE         8
HAPPY FROG          8
JUMBO AIR HOCKEY    8
SWEET FRENZY        7
TOY BOX             7
SPEED RIDER         6
DEEP SEA STORY      6
LUCKY CLAW          6
ZOOMOVE             5
UFO GALAXY          5
E-TICKET            5
FANTASTIC PARTY     5
SUPER BIKE          5
SAFARI RANGER       5
RAIMBOW             5
SLAM A WINNER       5
SPEED DRIVER        5
PO IT WIN           5
MOBIL LAMPU         4
RAINBOW SPINNER     4
Name: Game Title, dtype: int64

DEFINISI NORMALIZER (FINAL)

In [229]:
def normalize_game_title(x: str) -> str:
    if pd.isna(x):
        return x
    x = x.upper()
    x = re.sub(r"[^\w\s]", " ", x)
    x = re.sub(r"\s+", " ", x)
    return x.strip()


LOAD & BUILD AUDIT MERGE MAP (FINAL)

In [230]:
audit_df = pd.read_excel("audit_merge_map.xlsx")

audit_merge_map = dict(
    zip(
        audit_df["GT_B"].apply(normalize_game_title),
        audit_df["GT_A"].apply(normalize_game_title)
    )
)


APPLY KE DATA (FINAL APPLY)

In [231]:
df["GT_FINAL"] = df["GT_FINAL"].apply(normalize_game_title)
df["GT_FINAL"] = df["GT_FINAL"].replace(audit_merge_map)


VALIDASI WAJIB

In [232]:
remaining = set(audit_merge_map.keys()) & set(df["GT_FINAL"].unique())
remaining


{'E TICKET',
 'MAI MAI',
 'MAXIMUM TUNE 3',
 'MAXIMUM TUNE 3 DX',
 'PIRATES HOOK',
 'TROLEY'}

MANUAL MAPPING

In [233]:
manual_fix_map = {
    "E TICKET": "E-TICKET",
    "MAXIMUM TUNE 3": "MAXIMUM TUNE",
    "MAXIMUM TUNE 3 DX": "MAXIMUM TUNE",
    "TROLEY": "TROLLEY"
}


Apply manual fix

In [234]:
df["GT_FINAL"] = df["GT_FINAL"].replace(manual_fix_map)


VALIDASI FINAL (HARUS KOSONG)

In [235]:
{
    'E TICKET',
    'MAXIMUM TUNE 3',
    'MAXIMUM TUNE 3 DX',
    'TROLEY'
} & set(df["GT_FINAL"].unique())


set()

In [236]:
remaining = set(final_merge_map.keys()) & set(df["GT_FINAL"].unique())
remaining


{'MAI MAI', 'PIRATES HOOK', 'TROLLEY'}

APAKAH STRING-NYA BENAR-BENAR IDENTIK?

In [237]:
for x in remaining:
    print(repr(x))


'MAI MAI'
'TROLLEY'
'PIRATES HOOK'


EKSPOR DF HASIL NORMALISASI

In [238]:
print("Kolom df:")
print(df.columns.tolist())

print("\nKolom df_clean:")
print(df_clean.columns.tolist())


Kolom df:
['Center', 'Kode Game', 'Game Title', 'Keterangan', 'Kategori Game', 'Jumlah Diaktifkan', 'Kredit yg Digunakan', 'Bonus yg Digunakan', 'Total', 'Bulan', 'Tahun', 'Game Title Clean', 'GT_FINAL']

Kolom df_clean:
['Center', 'Kode Game', 'Game Title', 'Keterangan', 'Kategori Game', 'Jumlah Diaktifkan', 'Kredit yg Digunakan', 'Bonus yg Digunakan', 'Total', 'Bulan', 'Tahun', 'Game Title Clean', 'GT_FINAL']


In [239]:
(df['GT_FINAL'] == df_clean['GT_FINAL']).value_counts()


GT_FINAL
True    63467
Name: count, dtype: int64

In [240]:
df_clean['GT_FINAL'].value_counts().sum()

np.int64(63467)

BUAT TANGGAL

In [241]:
# Pastikan Bulan sudah title-case
df_clean['Bulan'] = df_clean['Bulan'].astype(str).str.title()

map_bulan = {
    'Januari': 1, 'Februari': 2, 'Maret': 3, 'April': 4,
    'Mei': 5, 'Juni': 6, 'Juli': 7, 'Agustus': 8,
    'September': 9, 'Oktober': 10, 'November': 11, 'Desember': 12
}

df_clean['Bulan_Angka'] = df_clean['Bulan'].map(map_bulan)

# Buat kolom Tanggal (tanggal 1 tiap bulan)
df_clean['Tanggal'] = pd.to_datetime(
    dict(
        year=df_clean['Tahun'],
        month=df_clean['Bulan_Angka'],
        day=1
    )
)

# Validasi
print("Kolom sekarang:")
print(df_clean.columns.tolist())

print("Range tanggal:")
print(df_clean['Tanggal'].min(), "→", df_clean['Tanggal'].max())


Kolom sekarang:
['Center', 'Kode Game', 'Game Title', 'Keterangan', 'Kategori Game', 'Jumlah Diaktifkan', 'Kredit yg Digunakan', 'Bonus yg Digunakan', 'Total', 'Bulan', 'Tahun', 'Game Title Clean', 'GT_FINAL', 'Bulan_Angka', 'Tanggal']
Range tanggal:
2024-01-01 00:00:00 → 2025-11-01 00:00:00


LANGKAH FINAL YANG WAJIB DIKUNCI (BIAR STABIL)

In [242]:
df_clean = df.copy()


EKSPOR DATA MESIN

In [243]:
cols_export = [
    'Center',
    'GT_FINAL',
    'Kode Game',
    'Keterangan',
    'Kategori Game',
    'Jumlah Diaktifkan',
    'Kredit yg Digunakan',
    'Tanggal'
]

df_clean[cols_export].to_excel(
    "CLEAN_DATA_MESIN_FINAL.xlsx",
    index=False
)
